<a href="https://colab.research.google.com/github/FireStrings/MasterDegree/blob/main/LSTM/Old/LSTMV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

import os
os.chdir("/content/drive/My Drive/UFSCAR/Notebooks")

Mounted at /content/drive/


### Imports

In [ ]:
%run UtilsNew.ipynb

In [ ]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import MeanSquaredError, MeanAbsoluteError
from tensorflow.keras.metrics import RootMeanSquaredError, R2Score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.layers import BatchNormalization, Input, Dense, InputLayer, LSTM, Bidirectional
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, normalize, StandardScaler
from sklearn.metrics import r2_score, mean_squared_error

### Funções

In [ ]:
def create_lstm_model(janela_tempo, input_name, with_compiled, with_bilstm):

    input_rad = Input(shape=(janela_tempo, 1), name="input_radiacao")
    if with_bilstm:
        x_lstm = Bidirectional(LSTM(64, return_sequences=True))(input_rad)
        x_lstm = Bidirectional(LSTM(32))(x_lstm)
    else:
        x_lstm = LSTM(64, return_sequences=True)(input_rad)
        x_lstm = LSTM(32)(x_lstm)

    if with_compiled:
        x_lstm = Dense(1, name="saida", activation='linear')(x_lstm)
        output = BatchNormalization()(x_lstm)

        model = Model(inputs=[input_rad], outputs=output)
        model.compile(optimizer='adam', loss='mse')

    else:
        model = x_lstm

    return model, input_rad


def create_lstm_model_old(janela_tempo, input_name, with_bilstm):

    model = Sequential()
    _input = InputLayer(input_shape=(janela_tempo, 1), name=input_name)
    model.add(_input)

    if with_bilstm:
        model.add(Bidirectional(LSTM(64, return_sequences=True)))
        model.add(Bidirectional(LSTM(32)))
    else:
        model.add(LSTM(64, return_sequences=True))
        model.add(LSTM(32))

    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='linear'))

    model.compile(optimizer='adam', loss='mse')

    return model, _input

def train_model(model, X, y, epochs=100):
    model.fit(X, y, epochs=epochs, batch_size=32, verbose=0)

    return model

def process_lstm(times, station, train_df, test_df, results, with_compiled, with_bilstm, with_plot):
    nn_type = "BiLSTM" if with_bilstm else "LSTM"
    train_input, train_output = data_to_input_and_output_lstm(train_df)
    test_input, test_output = data_to_input_and_output_lstm(test_df)

    for i in range(times):
        print("Starting process " + str(i) + " for station " + station + " with " + nn_type + " type")
        results[station][i] = {"train_pred":None, "test_pred": None, "period": None, "metrics":None}

        model, _ = create_lstm_model(janela_tempo, "input_radiacao", with_compiled, with_bilstm)
        train_model(model, train_input, train_output)

        train_pred = model.predict(train_input, verbose=0)
        test_pred = model.predict(test_input, verbose=0)

        if with_plot:
            period = anual_df[train_pred.shape[0]:]['data'].iloc[:test_pred.shape[0]]
            plot_results(test_pred.reshape(test_pred.shape[0]), test_output, period)

        results[station][i]["train_pred"] = train_pred
        results[station][i]["test_pred"] = test_pred
        results[station][i]["metrics"] = get_metrics(test_pred, test_output)